In [1]:
from dolfinx import geometry, fem, mesh, plot, io
from mpi4py import MPI
from petsc4py import PETSc
from petsc4py.PETSc import ScalarType
from ufl import (TrialFunction, Measure, TestFunction, dx, ds, grad, div, Dx, inner, lhs, rhs)

import timeit
import numpy as np
import pandas as pd

from utils.dolfinx import BoundaryCondition, generate_boundary_measure, eval_pointvalues, project
from meshes import pipe_0_2d, pipe_0_3d
from meshes import pipe_builder
# from utils.plotting import Mpl2DPlotter, Mpl2DAnimator

from IPython.display import Video
from pathlib import Path

In [2]:
save_dir = "/root/Meshes/pipe-model"
Path(save_dir).mkdir(parents=True, exist_ok=True)

U_PATH = f"{save_dir}/uj.xdmf"
PRESSURE_PATH = f"{save_dir}/pressure.xdmf"

In [3]:
def problem_setup(config):    
    """
    Perform problem configuration w.r.t. given parameters.
    Add new information to config.
    
    Parameters
    ----------
    config : dict
        Configuration parameters required for problem setup
    """
    
    # ---------------------------------------------------------------
    #               Mesh and function space definition
    # ---------------------------------------------------------------
    domain, mt, ft = pipe_builder.build_mesh(config['pipe'], MPI.COMM_WORLD, 0, config['logs'])
    dx = Measure('dx', subdomain_data=mt, domain=domain)
    
    V = fem.VectorFunctionSpace(domain, ("CG", 2))

    u = TrialFunction(V)
    v = TestFunction(V)

    # ---------------------------------------------------------------
    #        Definition physical characteristics functions
    # ---------------------------------------------------------------
    fluids = pd.read_csv('../data/physical_properties.csv', sep=';', index_col='Fluid')
    
    ro, c, eta = fluids.loc[[config['problem']['fluid'], 
                             config['problem']['contaminant']], 
                            ['Density', 'Speed of sound', 'Viscosity']].T.values

    # ---------------------------------------------------------------
    #               Construction of problem form
    # ---------------------------------------------------------------
    GAMMA, BETA = 0.5, 0.5
    dt = config['problem']['dt']

    mm = ScalarType(ro[0]) * inner(u, v) * dx(1)
    aa = ScalarType(ro[0] * c[0]**2) * inner(grad(u), grad(v)) * dx(1)
    cc = ScalarType(4./3 * eta[0]) * inner(grad(u), grad(v)) * dx(1)

    if sum(config['pipe']['bubble_lvl']) > 0.0:
        mm += ScalarType(ro[1]) * inner(u, v) * dx(2)
        aa += ScalarType(ro[1] * c[1]**2) * inner(grad(u), grad(v)) * dx(2)
        cc += ScalarType(4./3 * eta[1]) * inner(grad(u), grad(v)) * dx(2)
    
    F = mm + (dt * GAMMA * cc) + (0.5 * dt**2 * BETA * aa)
    
    # ---------------------------------------------------------------
    #               Definition of boundary conditions
    # ---------------------------------------------------------------
    measure = generate_boundary_measure([], domain, ft)
    
    u_D = lambda x: [x[0] * 0.0, x[1] * 0.0, x[2] * 0.0]
    u_N1 = fem.Constant(domain, ScalarType((config['problem']['pressure'], 0, 0)))
    u_N2 = fem.Constant(domain, ScalarType((0, 0, 0)))

    bcs = [BoundaryCondition("Dirichlet", 3, u_D, V, u, v, measure).bc,
           BoundaryCondition("Dirichlet", 2, u_D, V, u, v, measure).bc]
     
    nbcs = [BoundaryCondition("Neumann", 1, u_N1, V, u, v, measure).bc,
            BoundaryCondition("Neumann", 1, u_N2, V, u, v, measure).bc,
            BoundaryCondition("Neumann", 2, u_N2, V, u, v, measure).bc] 
    
    # ---------------------------------------------------------------
    #                      Update config
    # ---------------------------------------------------------------
    config['problem']['function_space'] = {
        'domain': domain,
        'V': V,
        'u': u,
        'v': v
    }
    config['problem']['physical_properties'] = {
        'ro': ro, 
        'c': c, 
        'eta': eta
    }
    config['problem']['forms'] = {
        'M': mm,
        'A': aa,
        'C': cc,
        'F': F
    }
    config['problem']['boundary_conditions'] = {
        'Dirichlet': bcs,
        'Neumann': nbcs
    }

In [4]:
def solve_problem(config):
    # ----------------------------------------------------------------
    #                   Obtaining the required data
    # ----------------------------------------------------------------

    domain, V = config['problem']['function_space']['domain'], \
                config['problem']['function_space']['V']

    mm, aa, cc, F = config['problem']['forms']['M'], \
                    config['problem']['forms']['A'], \
                    config['problem']['forms']['C'], \
                    config['problem']['forms']['F']

    bcs, nbcs = config['problem']['boundary_conditions']['Dirichlet'], \
                config['problem']['boundary_conditions']['Neumann']
    
    ro, c = config['problem']['physical_properties']['ro'][0], \
            config['problem']['physical_properties']['c'][0],
    
    T, dt = config['problem']['T'], \
            config['problem']['dt']

    # ----------------------------------------------------------------
    #              Configuring path for storing results
    # ----------------------------------------------------------------

    if config['file_prefix']:
        PREFIX = f"N_{config['pipe']['N']}_t_{config['pipe']['type']}" \
                 f"_br_{str(config['pipe']['bubble_radius']).split('.')[1]}" \
                 f"_bl_{'_'.join(str(int(pct * 100)) for pct in config['pipe']['bubble_lvl'])}_"
    else:
        PREFIX = ''
    
    U_PATH = f"{save_dir}/{PREFIX}uj.xdmf"
    PRESSURE_PATH = f"{save_dir}/{PREFIX}pressure.xdmf"

    xdmf_u, xdmf_p = None, None
    if config['save_to_file']:
        xdmf_u = io.XDMFFile(domain.comm, U_PATH, "w")
        xdmf_p = io.XDMFFile(domain.comm, PRESSURE_PATH, "w")
        xdmf_u.write_mesh(domain)
        xdmf_p.write_mesh(domain)
    
    # ----------------------------------------------------------------
    #                   Create initial condition
    # ----------------------------------------------------------------
    initial_condition = lambda x: [x[0] * 0.0, x[1] * 0.0, x[2] * 0.0]
    
    uj = fem.Function(V)
    uj_d = fem.Function(V)
    uj_dd = fem.Function(V)
    uj_d_d_tg = fem.Function(V)
    
    uj.interpolate(initial_condition)
    uj_d.interpolate(initial_condition)
    uj_dd.interpolate(initial_condition)
    uj_d_d_tg.interpolate(initial_condition)
    
    p = project(ro * c**2 * div(uj), domain, ("CG", 2))
    
    # ----------------------------------------------------------------
    #     Constructing left-hand side matrix and setuping solver
    # ----------------------------------------------------------------
    bilinear_form = fem.form(lhs(F))
    
    A = fem.petsc.assemble_matrix(bilinear_form, bcs=bcs)
    A.assemble()
    
    solver = PETSc.KSP().create(domain.comm)
    solver.setOperators(A)
    solver.setType(config['petsc']['solver'])
    solver.getPC().setType(config['petsc']['pc'])

    # ----------------------------------------------------------------
    #   Constructing right-hand side vectors for pulsing pressure
    # ----------------------------------------------------------------   
    GAMMA, BETA = 0.5, 0.5
    uj_d_d_tg.x.array[:] = dt * GAMMA * uj_d.x.array
        
    L1 = nbcs[0] + (cc * uj_d) + (aa * uj) + (aa * uj_d_d_tg)
    L2 = nbcs[1] + (cc * uj_d) + (aa * uj) + (aa * uj_d_d_tg)
    linear_form_1 = fem.form(rhs(L1))
    linear_form_2 = fem.form(rhs(L2))
    
    b1 = fem.petsc.create_vector(linear_form_1)
    b2 = fem.petsc.create_vector(linear_form_2)

    # ----------------------------------------------------------------
    #               Solving problem at each time step
    # ----------------------------------------------------------------    
    t = 0.0
    apply_t = config['problem']['pt']
    num_steps = int(T / dt)
    for i in range(num_steps):
        # ------------------------------------------------------------
        # Multiply values of uj_d by dt and GAMMA, 
        # because form does not support muliplication by constant
        # ------------------------------------------------------------
        uj_d_d_tg.x.array[:] = uj_d.x.array * dt * GAMMA

        # ------------------------------------------------------------
        #            Setting pulsing pressure at time `t`
        # ------------------------------------------------------------
        if apply_t > 0 and apply_t <= config['problem']['pt']:
            linear_form = linear_form_1         # pressure applied
            b = b1
        elif apply_t < 0:
            apply_t = config['problem']['delay']
            linear_form = linear_form_2         # pressure not applied
            b = b2
        else:
            linear_form = linear_form_2         # pressure not applied
            b = b2

        apply_t -= dt
        t += dt

        # ------------------------------------------------------------
        #       Assembling right-hand side vector at time `t`
        # ------------------------------------------------------------
        with b.localForm() as loc_b:
            loc_b.set(0)
        fem.petsc.assemble_vector(b, linear_form)

        # ------------------------------------------------------------
        #     Applying Dirichlet boundary condition to the vector
        # ------------------------------------------------------------
        fem.petsc.apply_lifting(b, [bilinear_form], [bcs])
        b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        fem.petsc.set_bc(b, bcs)

        # ------------------------------------------------------------
        #                   Solving linear problem
        # ------------------------------------------------------------
        if config['logs'] and i % config['problem']['save_step'] == 0:
            print(f'Info    : Solving problem (Step: {i})')
        
        solver.solve(b, uj_dd.vector)
        uj_dd.x.scatter_forward()

        # ------------------------------------------------------------
        #                     Updating solution
        # ------------------------------------------------------------
        uj1_d_array = uj_d.x.array + dt * uj_dd.x.array

        uj.x.array[:] = uj.x.array + 0.5 * dt * (uj_d.x.array + uj1_d_array)
        uj_d.x.array[:] = uj1_d_array

        # ------------------------------------------------------------
        #                 Save results into the files
        # ------------------------------------------------------------
        if i % config['problem']['save_step'] == 0:
            if config['save_to_file']:
                p = project(ro * c**2 * div(uj), domain, ("CG", 2))
                
                xdmf_u.write_function(uj, t)
                xdmf_p.write_function(p, t)
    
    # ------------------------------------------------------------
    #                 Closing results files
    # ------------------------------------------------------------
    if config['save_to_file']:
        xdmf_u.close()
        xdmf_p.close()
    
    if config['logs']:
        print(f'Info    : Done.')

In [5]:
config = {
    'pipe': {
        'N': 20,
        'type': 1,
        'bubble_radius': 0.05,
        'bubble_lvl': [0.0, 0.0]
    },
    'problem': {
        'fluid': 'Water',
        'contaminant': 'Air',
        'pressure': 1e3,
        'T':  1.7e-2,
        'dt': 8.75e-6,
        'pt': 8.75e-4,
        'delay': 3.5e-2,
        'save_step': 25
    },
    'petsc': {
        'solver': 'preonly',
        'pc': 'lu'
    },
    'logs': True,
    'file_prefix': True,
    'save_to_file': True
}

In [6]:
problem_setup(config)
solve_problem(config)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.00203966s, CPU 0.002226s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0747014s, CPU 0.074887s)
Info    : 1313 nodes 2628 elements
Info    : Solving problem (Step: 0)
Info    : Solving problem (Step: 25)
Info    : Solving problem (Step: 50)
Info    : Solving problem (Step: 75)
Info    : Solving problem (Step: 100)
Info    : Solving problem (Step: 125)
Info    : Solving problem (Step: 150)
Info    : Solving problem (Step: 175)
Info    : Solving problem (Step: 200)
Info    : Solving problem (Step: 225)
Info    : Solving problem (Step: 250)
Info    : Solving problem (Step: 275)
Info    : Solving problem (Step: 300)
Info    : Solving problem (Step: 325)
Info    : Solving problem (Step: 350)
Info    : Solving p